In [ ]:
!pip install codecarbon
!pip install thop
!pip install tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 504.2/504.2 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 77.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.4/386.4 kB 27.3 MB/s eta 0:00:00
  Attempting uninstall: termcolor
    Found existing installation: termcolor 2.5.0
    Uninstalling termcolor-2.5.0:
      Successfully uninstalled termcolor-2.5.0
  Attempting uninstall: prompt_toolkit
    Found existing installation: prompt_toolkit 3.0.48
    Uninstalling prompt_toolkit-3.0.48:
      Successfully uninstalled prompt_toolkit-3.0.48
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
i

In [ ]:
import time
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import models, datasets, transforms
from tqdm import tqdm
import codecarbon
from ptflops import get_model_complexity_info

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define dataset transforms
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Load datasets
full_train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

# Split dataset into training and validation sets
train_size = int(0.8 * len(full_train_dataset))
val_size = len(full_train_dataset) - train_size
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)

# Load the pre-trained teacher model (VGG-16)
teacher_model = models.vgg16(pretrained=False)
teacher_model.classifier[6] = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(4096, 10)
)
teacher_model.load_state_dict(torch.load("trained_vgg16_model.pth", map_location=device))
teacher_model = teacher_model.to(device)
teacher_model.eval()

# Define the student model (MobileNetV2)
student_model = models.mobilenet_v2(pretrained=True)
student_model.classifier[1] = nn.Sequential(
    nn.Linear(student_model.last_channel, 256),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, 10)
)
student_model = student_model.to(device)

# Define Dynamic Distillation Loss
class DynamicDistillationLoss(nn.Module):
    def __init__(self, initial_T=5.0, final_T=2.0, alpha=0.7, total_epochs=20):
        super(DynamicDistillationLoss, self).__init__()
        self.initial_T = initial_T
        self.final_T = final_T
        self.alpha = alpha
        self.total_epochs = total_epochs
        self.criterion_ce = nn.CrossEntropyLoss()

    def forward(self, student_logits, teacher_logits, labels, current_epoch):
        T = self.initial_T - (self.initial_T - self.final_T) * (current_epoch / self.total_epochs)
        distillation_loss = nn.KLDivLoss(reduction='batchmean')(
            torch.log_softmax(student_logits / T, dim=1),
            torch.softmax(teacher_logits / T, dim=1)
        ) * (T ** 2)
        student_loss = self.criterion_ce(student_logits, labels)
        return self.alpha * distillation_loss + (1 - self.alpha) * student_loss

criterion = DynamicDistillationLoss(initial_T=5.0, final_T=2.0, alpha=0.7, total_epochs=20)
optimizer = optim.AdamW(student_model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)

# Initialize CodeCarbon tracker
tracker = codecarbon.EmissionsTracker()
tracker.start()

# Training loop
epochs = 20
start_time = time.time()
best_val_accuracy = 0.0

for epoch in range(epochs):
    student_model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} - Training"):
        inputs, labels = inputs.to(device), labels.to(device)
        with torch.no_grad():
            teacher_outputs = teacher_model(inputs)
        student_outputs = student_model(inputs)
        loss = criterion(student_outputs, teacher_outputs, labels, current_epoch=epoch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(student_outputs, 1)
        correct_train += (predicted == labels).sum().item()
        total_train += labels.size(0)
    train_accuracy = 100 * correct_train / total_train
    scheduler.step()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss / len(train_loader):.4f}, Training Accuracy: {train_accuracy:.2f}%")

# Stop CodeCarbon tracker
emissions = tracker.stop()
end_time = time.time()
training_time = (end_time - start_time) / 60  # Convert to minutes

# Model Analysis
def get_model_size(model):
    torch.save(model.state_dict(), "temp_model.pth")
    size_mb = os.path.getsize("temp_model.pth") / (1024 * 1024)
    os.remove("temp_model.pth")
    return size_mb

def measure_inference_time(model, dataloader):
    model.eval()
    start_time = time.time()
    with torch.no_grad():
        for inputs, _ in dataloader:
            inputs = inputs.to(device)
            _ = model(inputs)
    end_time = time.time()
    return (end_time - start_time) / len(dataloader)

# Calculate FLOPs
flops, _ = get_model_complexity_info(student_model, (3, 32, 32), as_strings=False, print_per_layer_stat=False)
param_count = sum(p.numel() for p in student_model.parameters() if p.requires_grad)
model_size = get_model_size(student_model)
inference_time = measure_inference_time(student_model, test_loader)

# Print model analysis
print("\n--- Model Analysis ---")
print(f"Parameter Count: {param_count}")
print(f"Model Size: {model_size:.2f} MB")
print(f"FLOPs: {flops / 1e9:.2f} GFLOPs")
print(f"Training Time: {training_time:.2f} minutes")
print(f"Average Inference Time: {inference_time:.6f} seconds")
print("\n--- Energy and Emissions Report ---")
print(f"CO2 Emissions: {emissions:.6f} kg")


Using device: cuda
Files already downloaded and verified
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
<ipython-input-4-34310d872b55>:50: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions

Epoch [1/20], Loss: 5.9725, Training Accuracy: 55.92%


Epoch 2/20 - Training:  13%|█▎        | 82/625 [00:08<00:43, 12.41it/s][codecarbon INFO @ 10:42:43] Energy consumed for RAM : 0.000079 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 10:42:43] Energy consumed for all GPUs : 0.000728 kWh. Total GPU Power : 47.001471643617414 W
[codecarbon INFO @ 10:42:43] Energy consumed for all CPUs : 0.000709 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 10:42:43] 0.001517 kWh of electricity used since the beginning.
Epoch 2/20 - Training:  42%|████▏     | 260/625 [00:23<00:28, 12.62it/s][codecarbon INFO @ 10:42:58] Energy consumed for RAM : 0.000099 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 10:42:58] Energy consumed for all GPUs : 0.000914 kWh. Total GPU Power : 44.618649439633806 W
[codecarbon INFO @ 10:42:58] Energy consumed for all CPUs : 0.000886 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 10:42:58] 0.001899 kWh of electricity used since the beginning.
Epoch 2/20 - Training:  70%|███████   | 438/625 [00:38<00:13, 1

Epoch [2/20], Loss: 4.2548, Training Accuracy: 66.57%


...

Epoch [19/20], Loss: 1.6403, Training Accuracy: 77.06%


Epoch 20/20 - Training:  21%|██        | 129/625 [00:12<00:42, 11.57it/s][codecarbon INFO @ 10:58:59] Energy consumed for RAM : 0.001366 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 10:58:59] Energy consumed for all GPUs : 0.013771 kWh. Total GPU Power : 49.276042864736674 W
[codecarbon INFO @ 10:58:59] Energy consumed for all CPUs : 0.012224 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 10:58:59] 0.027361 kWh of electricity used since the beginning.
Epoch 20/20 - Training:  48%|████▊     | 301/625 [00:27<00:25, 12.51it/s][codecarbon INFO @ 10:59:14] Energy consumed for RAM : 0.001385 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 10:59:14] Energy consumed for all GPUs : 0.013972 kWh. Total GPU Power : 48.13494322496946 W
[codecarbon INFO @ 10:59:14] Energy consumed for all CPUs : 0.012401 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 10:59:14] 0.027758 kWh of electricity used since the beginning.
Epoch 20/20 - Training:  76%|███████▌  | 476/625 [00:42<00:11

Epoch [20/20], Loss: 1.4894, Training Accuracy: 77.43%


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(



--- Model Analysis ---
Parameter Count: 2554378
Model Size: 9.98 MB
FLOPs: 0.01 GFLOPs
Training Time: 17.98 minutes
Average Inference Time: 0.021753 seconds

--- Energy and Emissions Report ---
CO2 Emissions: 0.013418 kg


In [ ]:
pip install ptflops

In [ ]:
# Evaluate on the test dataset
student_model.eval()
correct_test, total_test = 0, 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = student_model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        correct_test += (predicted == labels).sum().item()
        total_test += labels.size(0)

test_accuracy = 100 * correct_test / total_test
print(f"Test Accuracy: {test_accuracy:.2f}%")


Test Accuracy: 80.92%
